<a href="https://colab.research.google.com/github/jorgemarpa/CHILE/blob/master/colab_WGAN_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
main_path = '/content/drive/My Drive/Colab_Notebooks'
print(main_path)

Mounted at /content/drive/
/content/drive/My Drive/Colab_Notebooks


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataloaders.py to dataloaders.py
Saving models.py to models (1).py
Saving training_v2.py to training_v2 (1).py
Saving training.py to training.py
Saving utils.py to utils (1).py
User uploaded file "dataloaders.py" with length 6751 bytes
User uploaded file "models.py" with length 6882 bytes
User uploaded file "training_v2.py" with length 10507 bytes
User uploaded file "training.py" with length 8146 bytes
User uploaded file "utils.py" with length 1634 bytes


# How to use TensorBoard in Colab

## Install TensorboardX

In [0]:
!pip install tensorboardX

    100% |████████████████████████████████| 133kB 11.3MB/s 


In [0]:
from tensorboardX import SummaryWriter

## Download and unzip ngrok

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-24 18:40:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.9.93, 52.202.60.111, 35.173.3.255, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.9.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.30M  15.5MB/s    in 0.9s    

2019-04-24 18:40:07 (15.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


## Run TensorBoard

In [4]:
LOG_DIR = './logs'
print(LOG_DIR)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# run ngrol
get_ipython().system_raw('./ngrok http 6006 &')
# get url
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

./logs
http://bc20d41c.ngrok.io


In [5]:
!ps aux | grep tensorboard

root        1505  0.1  1.9 1640168 263700 ?      Sl   18:43   0:02 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./logs --host 0.0.0.0 --port 6006
root        2217  0.0  0.0  39196  6464 ?        S    19:13   0:00 /bin/bash -c ps aux | grep tensorboard
root        2219  0.0  0.0  38568  5556 ?        S    19:13   0:00 grep tensorboard


# Install PyTorch 

In [0]:
!pip install -q torch==1.0.0 torchvision

    100% |████████████████████████████████| 591.8MB 24kB/s 


In [6]:
import torch
print(torch.__version__)

# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

1.0.0
CUDA is available!  Training on GPU ...


# Main Program

## Imports

In [0]:
import sys
import datetime
import torch
import torch.optim as optim
from dataloaders import *
from models import *
from training_v2 import Trainer

## Config

In [8]:
data = 'mnist'
num_samples = 28000
seq_len = 100
D_units = 32
G_units = 32
num_layers = 1
dropout = 0.2
bidirect = False
smooth_labels = False
latent_dim = 5
norm = False
learning_rate = 1e-4
betas = (.9, .99)
GP = True
gp_weight = 10.
decay = None
batch_size = 35
num_epochs = 200
D_rounds = 5
G_rounds = 1
use_time = True
use_err = True
shuffle = True
vis_freq = 5
n_feat = 3   # number of generated features = 3[time, mag, err]*num_signals
total_iter = num_epochs * int(num_samples*.6 / batch_size)

if use_time and use_err:
    n_feat = 3
elif use_time and not use_err:
    n_feat = 2
elif not use_time and use_err:
    n_feat = 2
elif not use_time and not use_err:
    n_feat = 1

if data in ['eros','synt']:
    model_name = 'WGAN-GP_%s_nfeat%i' % (data, n_feat)
    log_dir = ('./logs/%s/%s_units%i_drop%i_nlayers%i_Dr%i_norm%s_lr%.0e_ld%i/run_%s' %
                 (model_name, 'lstm', D_units,
                  int(dropout*100), num_layers, D_rounds, str(norm)[0],
                  learning_rate, latent_dim,
                  datetime.datetime.now().strftime("%y%m%d_%I%M")))
elif data == 'mnist':
    model_name = 'WGAN-GP_%s_%ix%i' % (data, 32, 32)
    log_dir = ('./logs/%s/%s_dim%i_lr%.0e_ld%i/run_%s' %
                 (model_name, 'cnn', 16,
                  learning_rate, 100,
                  datetime.datetime.now().strftime("%y%m%d_%I%M")))

print(model_name)
print(log_dir)

WGAN-GP_mnist_32x32
./logs/WGAN-GP_mnist_32x32/cnn_dim16_lr1e-04_ld100/run_190424_0714


## Load Data

In [9]:
if data == 'eros':
    data_loader, meta = get_eros_dataloaders(batch_size=batch_size, colab=True,
                                             use_time=use_time, use_err=use_err,
                                             shuffle=shuffle, norm=norm)
elif data == 'synt':
    data_loader, meta = get_synt_dataloaders(batch_size=batch_size, colab=True,
                                             use_time=use_time, use_err=use_err,
                                             shuffle=shuffle, norm=norm)
elif data == 'mnist':
    batch_size = 64
    data_loader, _ = get_mnist_dataloaders(batch_size=batch_size, colab=True)
else:
    print('Wrong dataset...')

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:04, 2420998.36it/s]                             


Extracting /content/drive/My Drive/Colab_Notebooks/data//MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129014.68it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/drive/My Drive/Colab_Notebooks/data//MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1523025.35it/s]                            
0it [00:00, ?it/s]

Extracting /content/drive/My Drive/Colab_Notebooks/data//MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 47242.09it/s]            


Extracting /content/drive/My Drive/Colab_Notebooks/data//MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## Define GAN Model, Ops, and Train

In [0]:
if data in ['eros', 'synt']:
    generator = Generator_LSTM(latent_dim, seq_len, G_units, batch_size,
                               num_layers=num_layers, dropout=dropout,
                               bidirectional=bidirect, output_dim=n_feat,
                               return_norm=False)
    discriminator = Discriminator_LSTM(n_feat, D_units, batch_size,
                                       num_layers=num_layers, dropout=dropout,
                                       bidirectional=False, output_dim=1)
else:
    img_size = (32, 32, 1)
    generator = Generator(img_size=img_size, latent_dim=100, dim=16)
    discriminator = Discriminator(img_size=img_size, dim=16)

print(generator)
print(discriminator)
# Initialize optimizers

G_optimizer = optim.Adam(generator.parameters(), lr=learning_rate,
                         betas=betas)
D_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate,
                         betas=betas)

print(G_optimizer)
print(D_optimizer)

# Train model
trainer = Trainer(generator, discriminator, G_optimizer, D_optimizer, batch_size,
                  GP=GP, gp_weight=gp_weight, critic_iterations=D_rounds,
                  print_every=25, log_dir=log_dir, dataname=data,
                  use_cuda=torch.cuda.is_available())

trainer.train(data_loader, num_epochs)

# Save models
torch.save(trainer.G.state_dict(), './gen_' + model_name + '.pt')
torch.save(trainer.D.state_dict(), './dis_' + model_name + '.pt')


Generator(
  (latent_to_features): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
  )
  (features_to_image): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(16, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): Sigmoid()
  )
)
Discriminator(
  (image_to_features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0